In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
gdrive_kaggle_wl_ar_sl = user_secrets.get_secret("gdrive_kaggle_wl_ar_sl")

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

def upload_to_gdrive(file_name, file_content):
    file_metadata = {
        'title': file_name,
        'parents': [{'id': gdrive_kaggle_wl_ar_sl}]
    }
    file = drive.CreateFile(file_metadata)
    file.SetContentString(file_content)
    file.Upload() # Files.insert()

In [234]:
os.listdir('/kaggle/working/karsl-kps/all_kps/03-train')

['0002', '0003', '0001', '0003.npz']

In [ ]:
def upload_to_gdrive(file_name, file_content):
    file_metadata = {
        'title': file_name,
        'parents': [{'id': gdrive_kaggle_wl_ar_sl}]
    }
    file = drive.CreateFile(file_metadata)
    file.SetContentString(file_content)
    file.Upload() # Files.insert()

In [1]:
!export TF_CPP_MIN_LOG_LEVEL=2
!pip install opencv-python mediapipe sklearn matplotlib
!wget https://github.com/issamjebnouni/Arabic-Word-level-Sign-Language-Recognition/raw/refs/heads/main/KARSL-502_Labels.xlsx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 51.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 12.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.8.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
google-api-core 1.34.1 requires protobuf!=3.20.0,!=3.20.1,!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<4.0.0dev,>=3.19.5, but you have protobuf 4.25.8 which is incompatible.
pandas-gbq 0.29.1 requires google-api-core<3.0.0,>=2.10.2, but you have google-api-core 1.34.1 which is incompatible.
google-cloud-storage 2.19.0 requires google-api-core<3.0.0dev,>=2.15.0, but you have google-ap

In [181]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm.notebook import tqdm
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # '2' suppresses warnings and info messages
os_join = os.path.join

DATA_DIR = "/kaggle/input/karsl-502"
KPS_DIR = "/kaggle/working/karsl-kps"

mp_hands = mp.solutions.hands
mp_pose = mp.solutions.pose
mp_face = mp.solutions.face_mesh

mp_face_nose_idx = sorted(mp.solutions.face_mesh_connections.FACEMESH_NOSE)[0][0]
mp_hand_wrist_idx = mp.solutions.hands.HandLandmark.WRIST
mp_pose_nose_idx = mp.solutions.pose.PoseLandmark.NOSE

pose_kps_idx = tuple(
    (
        mp.solutions.pose.PoseLandmark.LEFT_SHOULDER,
        mp.solutions.pose.PoseLandmark.RIGHT_SHOULDER,
        mp.solutions.pose.PoseLandmark.LEFT_ELBOW,
        mp.solutions.pose.PoseLandmark.RIGHT_ELBOW,
        mp.solutions.pose.PoseLandmark.LEFT_WRIST,
        mp.solutions.pose.PoseLandmark.RIGHT_WRIST,
    )
)
face_kps_idx = tuple(
    sorted(
        set(
            point
            for edge in [
                *mp.solutions.face_mesh_connections.FACEMESH_CONTOURS,
                *mp.solutions.face_mesh_connections.FACEMESH_IRISES,
            ]
            for point in edge
        )
    )
)
hand_kps_idx = tuple(range(len(mp.solutions.hands.HandLandmark)))

POSE_NUM = len(pose_kps_idx)
FACE_NUM = len(face_kps_idx)
HAND_NUM = len(hand_kps_idx)

KP2SLICE = {
    "pose": slice(0, POSE_NUM),
    "face": slice(POSE_NUM, POSE_NUM + FACE_NUM),
    "rh": slice(POSE_NUM + FACE_NUM, POSE_NUM + FACE_NUM + HAND_NUM),
    "lh": slice(POSE_NUM + FACE_NUM + HAND_NUM, POSE_NUM + FACE_NUM + HAND_NUM * 2),
}
POSE_KPS2IDX = {kps: idx for idx, kps in enumerate(pose_kps_idx)}
FACE_KPS2IDX = {kps: idx for idx, kps in enumerate(face_kps_idx)}
HAND_KPS2IDX = {kps: idx for idx, kps in enumerate(hand_kps_idx)}
KPS2IDX = {"pose": POSE_KPS2IDX, "face": FACE_KPS2IDX, "hand": HAND_KPS2IDX}


# usage: use it to draw mediapipe connections with the kps loaded from `.npy`arrays
for u, v in list(mp.solutions.face_mesh_connections.FACEMESH_IRISES)[:3]:
    print(face_kps_idx[FACE_KPS2IDX[u]], face_kps_idx[FACE_KPS2IDX[v]])


475 476
477 474
469 470


In [16]:
def get_karsl_words_min_frames_cnt():
    in_dir = "/kaggle/input/karsl-502"
    words_frames = defaultdict(lambda: (0, None))
    for signer in tqdm(["01", "02", "03"], desc="signer"):
        signer_dir = os_join(in_dir, signer, signer)

        for split in tqdm(["train", "test"], desc="split", leave=False):
            split_dir = os_join(signer_dir, split)

            for word in tqdm(range(1, 503), desc="words", leave=False):
                frames = (999, None)
                word_dir = os_join(split_dir, f"{word:04}")

                for rep in os.listdir(word_dir):
                    frames_dir = os_join(word_dir, rep)
                    frames_cnt = len(os.listdir(frames_dir))
                    if frames_cnt < frames[0]:
                        frames = (frames_cnt, frames_dir)

                if frames[0] > words_frames[word][0]:
                    words_frames[word] = frames
    return words_frames


# words_frames = get_karsl_words_min_frames_cnt()

In [ ]:
# !tar -cf sample.tar.gz '/kaggle/input/karsl-502/03/03/test/0102/03_03_0102_(22_12_16_10_40_19)_c'
# sorted(words_frames.values())

In [27]:
bad_samples = [
    # this sample has >260 frames, and after inspection it has many unrelated frames, so just drop it
    'karsl-502/02/02/train/0443/03_02_0443_(15_11_17_15_52_07)_c',
]

PAD_TKN = -1
SEQ_LEN = 80

In [1]:
def extract_frame_keypoints(frame, pose_model, face_model, hands_model):
    # TODO: normalize(?) keypoints after adjustment

    def get_xyz(lm):
        return (lm.x, lm.y, lm.z)

    # define numpy views, pose -> face -> rh -> lh
    all_kps = np.zeros((184, 3))  # (pose=6 + face=136 + rh+lh=42), xyz=3
    pose_kps = all_kps[KP2SLICE["pose"]]
    face_kps = all_kps[KP2SLICE["face"]]
    rh_kps = all_kps[KP2SLICE["rh"]]
    lh_kps = all_kps[KP2SLICE["lh"]]
    np_xyz = np.dtype((float, 3))

    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    def get_pose():
        nonlocal pose_kps
        results = pose_model.process(image_rgb)
        if results.pose_landmarks is None:
            return

        lms = results.pose_landmarks.landmark
        pose_kps[:] = np.fromiter(((lms[idx].x, lms[idx].y, lms[idx].z) for idx in pose_kps_idx), dtype=np_xyz)
        # pose_kps -= pose_kps[mp_pose_nose_idx]

    def get_face():
        nonlocal face_kps
        results = face_model.process(image_rgb)
        if results.multi_face_landmarks is None:
            return

        lms = results.multi_face_landmarks[0].landmark
        face_kps[:] = np.fromiter(((lms[idx].x, lms[idx].y, lms[idx].z) for idx in face_kps_idx), dtype=np_xyz)
        # face_kps -= face_kps[mp_face_nose_idx]

    def get_hands():
        nonlocal rh_kps, lh_kps
        results = hands_model.process(image_rgb)
        if results.multi_hand_landmarks is None:
            return

        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            if results.multi_handedness[i].classification[0].index == 0:
                lms = hand_landmarks.landmark
                rh_kps[:] = np.fromiter(((lm.x, lm.y, lm.z) for lm in lms), dtype=np_xyz)
                # rh_kps -= rh_kps[mp_hand_wrist_idx]
            else:
                lms = hand_landmarks.landmark
                lh_kps[:] = np.fromiter(((lm.x, lm.y, lm.z) for lm in lms), dtype=np_xyz)
                # lh_kps -= lh_kps[mp_hand_wrist_idx]

    with ThreadPoolExecutor(max_workers=3) as executor:
        executor.submit(get_pose)
        executor.submit(get_face)
        executor.submit(get_hands)
    
    return all_kps


def store_keypoint_arrays(models, word_dir, out_dir, split, signer, word):
    [pose_model, face_model, hands_model] = models

    all_kps = []
    videos_bar = tqdm(os.listdir(word_dir), leave=False)
    for video in videos_bar:
        video_dir = os_join(word_dir, video)
        videos_bar.set_description(f"Current video: {video}")

        video_kps = []
        for frame in sorted(os.listdir(video_dir)):
            frame = cv2.imread(os_join(video_dir, frame))
            video_kps.append(
                extract_frame_keypoints(frame, pose_model, face_model, hands_model)
            )

        pose_model.reset()
        face_model.reset()
        hands_model.reset()

        all_kps.append(video_kps.copy())

    word_kps_path = os_join(out_dir, "all_kps", f"{signer}-{split}", word)
    os.makedirs(os.dirname(word_kps_path), exist_ok=True)
    np.savez(word_kps_path, keypoints=np.concatenate(all_kps, axis=0))

In [ ]:
def extract_keypoints_from_frames(data_dir, kps_dir, splits=None, signers=None, selected_words=None):
    pose_model = mp_pose.Pose()
    face_model = mp_face.FaceMesh(refine_landmarks=True)
    hands_model = mp_hands.Hands()
    models = [pose_model, face_model, hands_model]

    splits = splits or ["train", "test"]
    signers = signers or ["01", "02", "03"]
    selected_words = selected_words or [f"{w:04}" for w in range(1, 503)]
    words_bar = tqdm(selected_words)
    for word in words_bar:
        words_bar.set_description(f"Current word: {word}")
        signers_bar = tqdm(signers, leave=False)
        for signer in signers:
            signers_bar.set_description(f"Current signer: {signer}")
            splits_bar = tqdm(splits, leave=False)
            for split in splits:
                splits_bar.set_description(f"Current split: {split}")
                word_dir = os_join(data_dir, signer, signer, split, word)
                store_keypoint_arrays(word_dir, kps_dir, signer, split, selected_words, models)
    
extract_keypoints_from_frames(DATA_DIR, KPS_DIR)

W0000 00:00:1754330934.277912   33771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


/kaggle/input/karsl-502/01/01/train:   0%|          | 0/499 [00:00<?, ?it/s]

  0%|          | 0/42 [00:00<?, ?it/s]

W0000 00:00:1754330934.302308   33764 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754330934.314759   33771 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754330934.339573   33764 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754330934.394689   33766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754330934.438299   33766 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754330935.165450   33770 inference_feedback_manager.cc:114] Feedback manager 

  0%|          | 0/68 [00:00<?, ?it/s]

...

W0000 00:00:1754332995.834084   33769 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754332995.881366   33769 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754332996.901115   33772 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754332996.912241   33765 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754332996.923974   33765 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754332996.941741   33771 inference_feedback_manager.cc:114] Feedback manager 

  0%|          | 0/42 [00:00<?, ?it/s]

W0000 00:00:1754333042.749289   33768 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333042.786453   33768 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333043.854470   33772 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333043.869767   33772 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333043.869997   33764 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333043.883965   33764 inference_feedback_manager.cc:114] Feedback manager 

  0%|          | 0/42 [00:00<?, ?it/s]

W0000 00:00:1754333089.555084   33768 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333089.592568   33769 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333090.518004   33773 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333090.534766   33764 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333090.535078   33773 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1754333090.548798   33765 inference_feedback_manager.cc:114] Feedback manager 

In [36]:
def load_keypoints(kps_dir, f_avg, split, words=None, signers=None):
    def pad_seq_(x, padding_amount):
        x = np.concatenate((x, np.repeat(x[-1], padding_amount, axis=0)), axis=0)

    signers = signers or ["01", "02", "03"]
    words = words or tuple((f"{v:04}" for v in range(1, 503)))

    kps_data_path = os_join(kps_dir, "all_kps")
    sequences = []
    for word in tqdm(words[:1]):
        for signer in signers:
            word_dir = os_join(kps_data_path, f"{signer}-{split}", word)
            sequences.append(
                [np.load(os_join(word_dir, video)) for video in os.listdir(word_dir)]
            )
    return sequences
    X = np.array(sequences)
    y = np.array([label_map[word] for word in words])
    y = OneHotEncoder(sparse=False).fit_transform(y.reshape(-1, 1))

    return X, y

# X, y = load_keypoints(KPS_DIR, SEQ_LEN, "train")
seq = load_keypoints(KPS_DIR, SEQ_LEN, "train")

  0%|          | 0/1 [00:00<?, ?it/s]

In [48]:
len(seq), len(seq[0]), len(seq[1]), len(seq[2]), seq[0][0].shape, seq[1][0].shape, seq[2][0].shape

(3, 1, 1, 1, (19, 184, 3), (23, 184, 3), (25, 184, 3))

In [203]:
np.concatenate(seq, axis=1).shape

(1, 67, 184, 3)

In [167]:
!tar -cf all-kps /kaggle/working/karsl-kps/all_kps

tar: Removing leading `/' from member names
